In [5]:
from pyspark.sql import SparkSession
import pandas as pd
from pymongo.mongo_client import MongoClient

In [2]:
spark = SparkSession.builder \
    .appName("Elhub MongoDB Upload") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

25/10/14 09:57:25 WARN Utils: Your hostname, Fabians-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.20.7.114 instead (on interface en0)
25/10/14 09:57:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/fabianheflo/.ivy2/cache
The jars for the packages stored in: /Users/fabianheflo/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-77133874-d36f-4717-8d31-62dbfd4b13c9;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.1 in central


:: loading settings :: url = jar:file:/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found org.apache.cassandra#java-driver-mapper-runtime;4.18.1 in central
	found org.apache.cassandra#java-driver-query-builder;4.18.1 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect;2.12.19 in central
:: resolution report :: resolve 224ms :: artifacts dl 8ms
	:: modules in use:
	com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal

✅ SparkSession started with Cassandra integration


### Reading production_silver table from Cassandra

In [3]:
silver_df = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_silver", keyspace="elhub_data")
    .load()
)

pdf = silver_df.toPandas()

### The dataset ready to be inserted into MongoDB

In [4]:
pdf

,pricearea,productiongroup,starttime,quantitykwh
0,NO5,hydro,2021-01-01 00:00:00,4068096.500
1,NO5,hydro,2021-01-01 01:00:00,4104306.000
2,NO5,hydro,2021-01-01 02:00:00,3948634.500
3,NO5,hydro,2021-01-01 03:00:00,3789087.200
4,NO5,hydro,2021-01-01 04:00:00,3911464.800
...,...,...,...,...
215348,NO1,wind,2021-12-31 19:00:00,124853.266
215349,NO1,wind,2021-12-31 20:00:00,108218.266
215350,NO1,wind,2021-12-31 21:00:00,108086.266
215351,NO1,wind,2021-12-31 22:00:00,121983.266


### MongoDB Connection


In [10]:
USR, PWD = open('../No_sync/MongoDB').read().splitlines()

# Build the URI using an f-string to avoid accidental TypeError when variables are None.
uri = ("mongodb+srv://{}:{}@ind320-cluster-fh.93gnbd4.mongodb.net/?retryWrites=true&w=majority&appName=IND320-cluster-fh")

# Create a new client and connect to the server
client = MongoClient(uri.format(USR, PWD))

database = client["elhub"]
collection = database["production_silver"]

### Insterting to MongoDB 

In [11]:
records = pdf.to_dict(orient='records')
collection.insert_many(records)
print(f"✅ {len(records)} records inserted into MongoDB.")

✅ 215353 records inserted into MongoDB.
